In [1]:
from pathlib import Path

Path.cwd()

import logging
import pickle
import os
import random
import shutil
import time

import numpy as np
import torch

from sklearn.model_selection import train_test_split

from timegan.models import TimeGAN
from timegan.models.utils import timegan_trainer, timegan_generator

/home/lukas/mambaforge/envs/timegan/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'models'